In [1]:
import torch
#clear memory better
with torch.no_grad():
    torch.cuda.empty_cache()
import numpy as np
import util.npose_util as nu
import os
import pathlib
import dgl
from dgl import backend as F
import torch_geometric
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Dict
from torch import Tensor
from dgl import DGLGraph
from torch import nn
# from chemical import cos_ideal_NCAC #from RoseTTAFold2
from torch import einsum
import time
torch.cuda.is_available()

True

In [2]:
import se3_diffuse.utils as du

In [3]:
from data_rigid_diffuser import so3_diffuser
from data_rigid_diffuser import r3_diffuser
from data_rigid_diffuser import oneHot_diffuser
from scipy.spatial.transform import Rotation
from data_rigid_diffuser import rigid_utils as ru
import yaml
from data_rigid_diffuser.diffuser import FrameDiffNoise
from gudiff_model import Data_Graph
from gudiff_model.Data_Graph import build_npose_from_coords, dump_coord_pdb, define_graph_edges, make_pe_encoding
from gudiff_model.Data_Graph import Helix4_Dataset, Make_KNN_MP_Graphs

In [4]:
from se3_transformer.model.basis import get_basis, update_basis_with_fused
from se3_transformer.model.transformer import Sequential, SE3Transformer
from se3_transformer.model.transformer_topk import SE3Transformer_topK
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from se3_transformer.model.layers.attentiontopK import AttentionBlockSE3
from se3_transformer.model.layers.linear import LinearSE3
from se3_transformer.model.layers.convolution import ConvSE3, ConvSE3FuseLevel
from se3_transformer.model.layers.norm import NormSE3
from se3_transformer.model.layers.pooling import GPooling, Latent_Unpool, Unpool_Layer
from se3_transformer.runtime.utils import str2bool, to_cuda
from se3_transformer.model.fiber import Fiber
from se3_transformer.model.transformer import get_populated_edge_features

In [5]:
#indices for, unsure if needed
CA = Data_Graph.CA
N = Data_Graph.N
C = Data_Graph.C

# #find better way to incorporate coord_scale

# #needed
# N_CA_dist = (Data_Graph.N_CA_dist/10.).to('cuda')
# C_CA_dist = (Data_Graph.C_CA_dist/10.).to('cuda')



In [6]:
# data_path_str  = 'data/h4_ca_coords.npz'
# test_limit = 1028
# rr = np.load(data_path_str)
# ca_coords = [rr[f] for f in rr.files][0][:test_limit,:,:3]
# ca_coords.shape

# getting N-Ca, Ca-C vectors to add as typeI features
#apa = apart helices for val/train split
#tog = together helices for val/train split

#mode for tablet
apa_path_str  = 'data_npose/h4_apa_coords.npz'
tog_path_str  = 'data_npose/h4_tog_coords.npz'

#grab the first 3 atoms which are N,CA,C
test_limit = 256
rr = np.load(apa_path_str)
coords_apa = [rr[f] for f in rr.files][0][:test_limit,:]

rr = np.load(tog_path_str)
coords_tog = [rr[f] for f in rr.files][0][:test_limit,:]

In [7]:
B = 32
L=65
limit = 128
h4_trainData = Helix4_Dataset(coords_tog[:limit])
h4_valData = Helix4_Dataset(coords_apa[:limit])
train_dL = DataLoader(h4_trainData, batch_size=B, shuffle=True, drop_last=True)
val_dL   = DataLoader(h4_valData, batch_size=B, shuffle=True, drop_last=True)

In [8]:
B = 32
L=65
limit = 128
prot_trainData = Data_Graph.ProteinBB_Dataset(coords_tog[:limit], n_nodes=128,
              n_atoms=5, coord_div=10, cast_type=torch.float32)
train_dL = DataLoader(prot_trainData, batch_size=B, shuffle=True, drop_last=True)

In [9]:
testiter = iter(train_dL)
bb_dict = next(testiter)

In [10]:
from data_rigid_diffuser import diffuser

In [11]:
fdn_an = FrameDiffNoise()

In [12]:
config_path='data_rigid_diffuser/base.yaml'

        


In [13]:
fnd = FrameDiffNoise(config_path)


In [14]:
noised_dict = fnd.forward(bb_dict)

In [15]:
noised_dict.keys()

dict_keys(['bb_noised', 't_vec', 'score_scales', 'edges_noised'])

In [19]:
def circular_pe_encoding(n_nodes=128,embed_dim=12, cast_type=torch.float32):
    #positional encoding of node, all repeat every n_nodes
    i_array = np.arange(1,(embed_dim/2)+1)
    period = 2*np.pi
    wk = period/(n_nodes)*i_array**2
    t_array = np.arange(n_nodes)
    si = torch.tensor(np.sin(wk*t_array.reshape((-1,1))))
    ci = torch.tensor(np.cos(wk*t_array.reshape((-1,1))))
    pe = torch.stack((si,ci),axis=2).reshape(t_array.shape[0],embed_dim).type(cast_type)
    
    return pe

In [81]:
KNN = 30
n_nodes = 6
cast_type = torch.float32
pe = circular_pe_encoding(n_nodes=n_nodes, cast_type=cast_type)
mp_stride = 4


In [82]:
src_start = torch.arange(n_nodes) #positive direction
dst_start = torch.roll(src_start,-1) #negative dir

In [83]:
xyz = torch.arange(n_nodes)[:,None].repeat((1,3))
xyz[-1] = torch.tensor([4,4,4])
xyz[0] = torch.tensor([1,1,1])

In [106]:
mask = torch.tensor([False,True,True,True,True,False])

In [168]:
def null_nodes_graph(xyz,real_mask, k=2):
    """Takes in array of XYZ coordinates with null nodes repeated at termini.
    
    Returns Graph that has a linear connections between all nodes indices"""
    
    n_nodes = xyz.shape[0]
    src_start = torch.arange(n_nodes) #positive direction
    dst_start = torch.roll(src_start,-1) #negative dir
    
    #ma
    real_nodes_src = src_start[real_mask==True]
    null_nodes_src = src_start[real_mask==False]
    real_nodes_dst = dst_start[real_mask==True]
    null_nodes_dst = dst_start[real_mask==False]

    g_knn=dgl.knn_graph(xyz[real_mask],k=k,exclude_self=True) # normal graph
    g_knn.add_nodes(len(null_nodes_src)) # null nodes to the end

    src_knn_shift = torch.zeros_like(src_start)
    dst_knn_shift = torch.zeros_like(dst_start)

    src_knn_shift[:len(real_nodes_src)] = real_nodes_src
    src_knn_shift[len(real_nodes_src):] = null_nodes_src
    dst_knn_shift[:len(real_nodes_src)] = real_nodes_dst
    dst_knn_shift[len(real_nodes_src):] = null_nodes_dst

    g_nodes_index = src_start#torch.arange(n_nodes) 
    src = g_nodes_index[src_knn_shift]
    dst = g_nodes_index[dst_knn_shift]
    
    #adjust knn indices to match original inputs with null nodes having indices
    knn_src, knn_dst = g_knn.edges()
    num_null_nodes_low =  len(null_nodes_src[null_nodes_src<len(real_nodes_src)])
    knn_src = knn_src + num_null_nodes_low
    knn_dst = knn_dst + num_null_nodes_low
    
    g = dgl.graph((knn_src,knn_dst))

    #can just add edges on top, simple to delete parallel; 
    g.add_edges(src,dst) 
    g.add_edges(dst,src)
    g = dgl.to_simple(g)

    return g

def decirc(graph_in):
    """Remove circular connections. Last node to first node"""
    src,dst = graph_in.edges()

    circfw = (src==graph_in.nodes().max()) & (dst==0) 
    circbw = (src==0) & (dst==graph_in.nodes().max())
    both_circ = circfw | circbw
    
    src_out = src[~both_circ]
    dst_out = dst[~both_circ]
    
    return dgl.graph((src_out,dst_out))

def recirc(bg, )

In [176]:
g = null_nodes_graph(xyz,mask)
g2 = null_nodes_graph(xyz,mask)
gb = dgl.batch((decirc(g),decirc(g)))

In [181]:
len(g.nodes())

6

In [180]:
gb.edges()

(tensor([ 0,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  5,  6,  7,  7,  8,  8,  8,
          9,  9,  9, 10, 10, 11]),
 tensor([ 1,  0,  2,  1,  3,  4,  1,  2,  4,  3,  5,  4,  7,  6,  8,  7,  9, 10,
          7,  8, 10,  9, 11, 10]))

In [212]:
def concat_monomer_graphs(graph_list):
    """Add single connection between each graph"""
    
    batched_graph = dgl.batch([decirc(g) for g in graph_list])
    src = torch.zeros((len(graph_list),),dtype=torch.int64) 
    dst = torch.zeros_like(src)
    
    #recirc concats
    nindex = 0
    src[nindex] = 0
    dst[nindex] = len(batched_graph.nodes())-1 #
    
    
    for i in range(1,len(graph_list)):
        print(i)
        src[i] = graph_list[nindex].nodes().max()+nindex
        dst[i] = src[i]+1
        nindex = src[i]
    
    print(src,dst)
    
    batched_graph.add_edges(src,dst)
    batched_graph.add_edges(dst,src)
    
    return batched_graph
    
    

In [213]:
gout=concat_monomer_graphs([g,g])

1
tensor([0, 5]) tensor([11,  6])


In [214]:
gout.edges()

(tensor([ 0,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  5,  6,  7,  7,  8,  8,  8,
          9,  9,  9, 10, 10, 11,  0,  5, 11,  6]),
 tensor([ 1,  0,  2,  1,  3,  4,  1,  2,  4,  3,  5,  4,  7,  6,  8,  7,  9, 10,
          7,  8, 10,  9, 11, 10, 11,  6,  0,  5]))

In [170]:
circfw = (src==g.nodes().max()) & (dst==0) 
circfw

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False,  True, False])

In [171]:
circbw = (src==0) & (dst==g.nodes().max())
circbw

tensor([False,  True, False, False, False, False, False, False, False, False,
        False, False, False, False])

In [173]:
decirc(g).edges()

(tensor([0, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5]),
 tensor([1, 0, 2, 1, 3, 4, 1, 2, 4, 3, 5, 4]))

tensor([0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5])

In [163]:
dst

tensor([1, 5, 0, 2, 1, 3, 4, 1, 2, 4, 3, 5, 0, 4])

In [ ]:
 circfw = (src!=g.nodes().max()) & (dst==0) 
circbw = (src==0) & (dst!=graph_in.nodes().max())
both_circ = circfw & circbw

In [151]:
decirc(g).edges()
    

(tensor([], dtype=torch.int64), tensor([], dtype=torch.int64))

In [119]:
gb = dgl.batch((g,g2))

In [120]:
gb.edges()

(tensor([ 0,  0,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,
          8,  8,  8,  9,  9,  9, 10, 10, 11, 11]),
 tensor([ 1,  5,  0,  2,  1,  3,  4,  1,  2,  4,  3,  5,  0,  4,  7, 11,  6,  8,
          7,  9, 10,  7,  8, 10,  9, 11,  6, 10]))

In [117]:
gn1 = g.nodes()
gn2 = g2.nodes()


src1,dst1 = g.edges()
src2,dst2 = g2.edges()

src1 = src1[src1!=g.nodes().max()]





In [116]:
gn2

tensor(5)

In [105]:
g.edges()

(tensor([0, 0, 1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5]),
 tensor([1, 5, 0, 2, 1, 3, 4, 2, 4, 2, 3, 5, 0, 4]))

In [ ]:
def concat_graphs(graph1, graph2):
    """Concatenate graphs: (dircularize and concat then recircularize)"""
    
    

In [97]:
mask

tensor([False,  True, False,  True,  True, False])

In [34]:
g.edges()

(tensor([0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5]),
 tensor([1, 5, 0, 2, 3, 0, 1, 3, 2, 4, 3, 5, 0, 4]))

In [20]:
class Make_KNN_MP_Graphs():
    
    #8 long positional encoding
    NODE_FEATURE_DIM_0 = 12
    EDGE_FEATURE_DIM = 1 # 0 or 1 primary seq connection or not
    NODE_FEATURE_DIM_1 = 2
    
    def __init__(self, mp_stride=4, n_nodes=65, radius=15, coord_div=10, cast_type=torch.float32, channels_start=32,
                       ndf1=6, ndf0=32,cuda=True):
        
        self.KNN = 30
        self.n_nodes = n_nodes
        self.pe = make_pe_encoding(n_nodes=n_nodes)
        self.mp_stride = mp_stride
        self.cast_type = cast_type
        self.channels_start = channels_start
        
        self.cuda = cuda
        self.ndf1 = ndf1 #awkard adding of nodes features to mpGraph
        self.ndf0 = ndf0
        
    def create_and_batch(self, bb_dict):
        
        graphList = []
        mpGraphList = []
        mpRevGraphList = []
        mpSelfGraphList = []
        
        for j, caXYZ in enumerate(bb_dict['CA']):
            graph = dgl.knn_graph(caXYZ, self.KNN)
            graph.ndata['pe'] = self.pe
            graph.ndata['pos'] = caXYZ
            graph.ndata['bb_ori'] = torch.cat((bb_dict['N_CA'][j],  bb_dict['C_CA'][j]),axis=1)
            
            #define covalent connections
            esrc, edst = graph.edges()
            graph.edata['con'] = (torch.abs(esrc-edst)==1).type(self.cast_type).reshape((-1,1))
            
            mp_list = torch.zeros((len(list(range(0,self.n_nodes, self.mp_stride))),caXYZ.shape[1]))
            
            new_src = torch.tensor([],dtype=torch.int)
            new_dst = torch.tensor([],dtype=torch.int)
            
            new_src_rev = torch.tensor([], dtype=torch.int)
            new_dst_rev = torch.tensor([], dtype=torch.int)
           
            i=0#mp list counter
            for x in range(0,self.n_nodes, self.mp_stride):
                src, dst = graph.in_edges(x) #dst repeats x
                n_tot = torch.cat((torch.tensor(x).unsqueeze(0),src)) #add x to node list
                mp_list[i] = caXYZ[n_tot].sum(axis=0)/n_tot.shape[0]
                mp_node = i + graph.num_nodes() #add midpoints nodes at end of graph
                #define edges between midpoint nodes and nodes defining midpoint for midpointGraph
                
                new_src = torch.cat((new_src,n_tot))
                new_dst = torch.cat((new_dst,
                                     (torch.tensor(mp_node).unsqueeze(0).repeat(n_tot.shape[0]))))
                #and reverse graph for coming off
                new_src_rev = torch.cat((new_src_rev,
                                         (torch.tensor(mp_node).unsqueeze(0).repeat(n_tot.shape[0]))))
                new_dst_rev = torch.cat((new_dst_rev,n_tot))
                
                i+=1
                
            mpGraph = dgl.graph((new_src,new_dst))
            mpGraph.ndata['pos'] = torch.cat((caXYZ,mp_list),axis=0).type(self.cast_type)
            mp_node_indx = torch.arange(0,self.n_nodes, self.mp_stride).type(torch.int)
            #match output shape of first transformer
            pe_mp = torch.cat((self.pe,torch.zeros((self.pe.shape[0],self.channels_start-self.pe.shape[1]))),axis=1)
            mpGraph.ndata['pe'] = torch.cat((pe_mp,pe_mp[mp_node_indx]))
            mpGraph.edata['con'] = torch.zeros((mpGraph.num_edges(),1))
            
            mpGraph_rev = dgl.graph((new_src_rev,new_dst_rev))
            mpGraph_rev.ndata['pos'] = torch.cat((caXYZ,mp_list),axis=0).type(self.cast_type)
            mpGraph_rev.ndata['pe'] = torch.cat((pe_mp,pe_mp[mp_node_indx]))
            mpGraph_rev.edata['con'] = torch.zeros((mpGraph_rev.num_edges(),1))
            
            #make graph for self interaction of midpoints
            v1,v2,edge_data, ind = define_graph_edges(len(mp_list))
            mpSelfGraph = dgl.graph((v1,v2))
            mpSelfGraph.edata['con'] = edge_data.reshape((-1,1))
            mpSelfGraph.ndata['pe'] = self.pe[mp_node_indx] #not really needed
            mpSelfGraph.ndata['pos'] = mp_list.type(self.cast_type)
            
            
            mpSelfGraphList.append(mpSelfGraph) 
            mpGraphList.append(mpGraph)
            mpRevGraphList.append(mpGraph_rev)
            graphList.append(graph)
        
        return dgl.batch(graphList), dgl.batch(mpGraphList), dgl.batch(mpSelfGraphList), dgl.batch(mpRevGraphList)
    
    def prep_for_network(self, bb_dict, cuda=True):
    
        batched_graph, batched_mpgraph, batched_mpself_graph, batched_mpRevgraph =  self.create_and_batch(bb_dict)
        
        edge_feats        =    {'0':   batched_graph.edata['con'][:, :self.EDGE_FEATURE_DIM, None]}
        edge_feats_mp     = {'0': batched_mpgraph.edata['con'][:, :self.EDGE_FEATURE_DIM, None]} #def all zero now
        edge_feats_mpself = {'0': batched_mpself_graph.edata['con'][:, :self.EDGE_FEATURE_DIM, None]}
#         edge_feats_mp     = {'0': batched_mpRevgraph.edata['con'][:, :self.EDGE_FEATURE_DIM, None]}
        batched_graph.edata['rel_pos']   = _get_relative_pos(batched_graph)
        batched_mpgraph.edata['rel_pos'] = _get_relative_pos(batched_mpgraph)
        batched_mpself_graph.edata['rel_pos'] = _get_relative_pos(batched_mpself_graph)
        batched_mpRevgraph.edata['rel_pos'] = _get_relative_pos(batched_mpRevgraph)
        # get node features
        node_feats =         {'0': batched_graph.ndata['pe'][:, :self.NODE_FEATURE_DIM_0, None],
                              '1': batched_graph.ndata['bb_ori'][:,:self.NODE_FEATURE_DIM_1, :3]}
        node_feats_mp =      {'0': batched_mpgraph.ndata['pe'][:, :self.ndf0, None],
                              '1': torch.ones((batched_mpgraph.num_nodes(),self.ndf1,3))}
        #unused
        node_feats_mpself =  {'0': batched_mpself_graph.ndata['pe'][:, :self.NODE_FEATURE_DIM_0, None]}
        
        if cuda:
            bg,nf,ef = to_cuda(batched_graph), to_cuda(node_feats), to_cuda(edge_feats)
            bg_mp, nf_mp, ef_mp = to_cuda(batched_mpgraph), to_cuda(node_feats_mp), to_cuda(edge_feats_mp)
            bg_mps, nf_mps, ef_mps = to_cuda(batched_mpself_graph), to_cuda(node_feats_mpself), to_cuda(edge_feats_mpself)
            bg_mpRev = to_cuda(batched_mpRevgraph)
            
            return bg,nf,ef, bg_mp, nf_mp, ef_mp, bg_mps, nf_mps, ef_mps, bg_mpRev
        
        else:
            bg,nf,ef = batched_graph, node_feats, edge_feats
            bg_mp, nf_mp, ef_mp = batched_mpgraph, node_feats_mp, edge_feats_mp
            bg_mps, nf_mps, ef_mps = batched_mpself_graph, node_feats_mpself, edge_feats_mpself
            bg_mpRev = batched_mpRevgraph
            
            return bg,nf,ef, bg_mp, nf_mp, ef_mp, bg_mps, nf_mps, ef_mps, bg_mpRev
        
            

def get_edge_features(graph,edge_feature_dim=1):
    return {'0': graph.edata['con'][:, :edge_feature_dim, None]}

def define_poolGraph(n_nodes, batch_size, cast_type=torch.float32, cuda_out=True ):
    
    v1,v2,edge_data, ind = define_graph_edges(n_nodes)
    #pe = make_pe_encoding(n_nodes=n_nodes)#pe e
    
    graphList = []
    
    for i in range(batch_size):
        
        g = dgl.graph((v1,v2))
        g.edata['con'] = edge_data.type(cast_type).reshape((-1,1))
        g.ndata['pos'] = torch.zeros((n_nodes,3),dtype=torch.float32)

        graphList.append(g)
        
    batched_graph = dgl.batch(graphList)
    
    if cuda_out:
        return to_cuda(batched_graph)
    else:
        return batched_graph

In [35]:
xyz

tensor([[1, 1, 1],
        [1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [4, 4, 4],
        [4, 4, 4]])

In [64]:
dst

tensor([2, 3, 4, 5, 1, 0])